### 6.06: GNOD final

* Check whether or not the input song is in the PopVortex Hot 100.
* Recommend another Hot 100 song
* If it is NOT in the hot 100, then
    * collect the audio features from the Spotify API for the input song.
    * You want to send the Spotify audio features of the submitted song to the clustering model, which should return a cluster number.
    * Then you recommend a song from the same cluster number.


### Authentification

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [2]:
secrets_file = open("secrets.txt","r")
string = secrets_file.read()
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        #print(line.split(':'))
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

## Retrieve files

In [6]:
from sklearn.cluster import KMeans
import numpy as np
from random import randint 

In [29]:
#load kmean model
import pickle
with open('kmeans_model_spotify.pkl', 'rb') as f: #rb is loading
    kmeans_loaded = pickle.load(f)

In [30]:
# load scaler
with open('scaler_spotify.pkl', 'rb') as f: 
    scaler_loaded = pickle.load(f)

In [16]:
#clusternumber=kmeans_loaded.cluster_centers_
#print(clusternumber)

In [ ]:
#vbigrepertoire=pd.read_csv('kaggle_songs_with_clusters.csv')

In [31]:
bigrepertoire=pd.read_csv('spotify_10000_with_clusters.csv')

In [3]:
repertoire=pd.read_csv('repertoire_top100.csv')

### Function

In [48]:

def recommendation():
    track_name=input("enter song title")
    artist_name=input("enter artist name")

#is song in TOP 100?
    if track_name in repertoire['song_title'].values:
        #index of input song if it exists
        input_index = repertoire[repertoire['song_title'] == track_name].index[0]
        
        # generate a random index different from the index of the input song
        random_index = random.randint(0, len(repertoire) - 2)
        
        if random_index >= input_index:
            random_index += 1
        random_track_100 = repertoire.loc[random_index, 'song_title']
        random_artist_100 = repertoire.loc[random_index, 'Artist_Name']
        print(f'We recommend you this Hot Song "{random_track_100}" by "{random_artist_100}". Do you like it?')

#is song on Spotify?
    else: 
        results = sp.search(q=f'track:{track_name} artist:{artist_name}', type='track', limit=1)
        # Extract track ID from spotify search
        if results['tracks']['items']: #if search is successful
            track_id = results['tracks']['items'][0]['id'] #we take ID from first instance of the search
            ##print(f'Track ID for "{track_name}" by {artist_name}: {track_id}') #this will show spotify search was successful
            #get audio features
            features=sp.audio_features(track_id)
            df_features=pd.DataFrame(features) #we make the features a df so we can transform it
            ##display(df_features)
            x=df_features[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 
            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']]
            #scale
            x_scaled=scaler_loaded.transform(x)
            #find cluster
            x_cluster=kmeans_loaded.predict(x_scaled)
            print(f'Cluster is:"{x_cluster}"')
            #cluster is a list so we would need to refer to index 0 : [cluster[0]]
            filtered= bigrepertoire[bigrepertoire['cluster'].isin(x_cluster)]#bigrepertoire[bigrepertoire['cluster'] == x_cluster]
            ##display(filtered)
            #find a random song within the same cluster
            filtered.reset_index(drop=True, inplace=True) #resetting index
            random_index = np.random.randint(0, len(filtered)) #you want to chose within cluster length
            reco_track = filtered.loc[random_index, 'song_name']
            reco_artist = filtered.loc[random_index, 'name']
            print(f'We recommend you a similar song: "{reco_track}" by "{reco_artist}". turn this up!')

        else:
            print(f'Oops. Track "{track_name}" by {artist_name} not found.')



In [50]:
recommendation()

enter song title Daddy Cool
enter artist name Boney M


Cluster is:"[7]"
We recommend you a similar song: "Another Love - Zwette Edit" by "Zwette". turn this up!
